In [1]:
#import the libraries
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
#Set the image size with are learning from
IMG_WIDTH, IMG_HEIGHT = 150,150

#Set the constants
TRAIN_DATA_DIR = 'Resources/chest_xray/train'
VALIDATION_DATA_DIR = 'Resources/chest_xray/test'
NB_TRAIN_SAMPLES = 200  #Must match number of files
NB_VALIDATION_SAMPLES = 200

In [3]:
EPOCHS = 50 #Higher for more time training model... diminishing returns
BATCH_SIZE = 5

In [4]:
if K.image_data_format() == 'channels_first':
        input_shape = (3, IMG_WIDTH, IMG_HEIGHT)
else:
        input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

    


In [5]:
train_datagen = ImageDataGenerator(
            rotation_range = 40,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            rescale = 1.0/255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
    
    # this is the augmentation configuration we will use for testing:
    # only rescaling
test_datagen = ImageDataGenerator(rescale= 1. / 255)

train_generator = train_datagen.flow_from_directory(
            TRAIN_DATA_DIR,
            target_size=(IMG_WIDTH, IMG_HEIGHT),
            batch_size=BATCH_SIZE,
            class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
            VALIDATION_DATA_DIR,
            target_size=(IMG_WIDTH, IMG_HEIGHT),
            batch_size=BATCH_SIZE,
            class_mode='binary')

Found 4010 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add( BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add( BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add( BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add( BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add( BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
    
   
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])

In [7]:
model.fit_generator(
            train_generator,
            steps_per_epoch=NB_VALIDATION_SAMPLES // BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=validation_generator,
            validation_steps=NB_VALIDATION_SAMPLES // BATCH_SIZE)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
40/40 [==============================] - 11s 284ms/step - loss: 0.8836 - accuracy: 0.7400 - val_loss: 1.7665 - val_accuracy: 0.6400
Epoch 2/50
40/40 [==============================] - 10s 253ms/step - loss: 0.5553 - accuracy: 0.8350 - val_loss: 4.3615 - val_accuracy: 0.5850
Epoch 3/50
40/40 [==============================] - 9s 226ms/step - loss: 0.4848 - accuracy: 0.8200 - val_loss: 7.0850 - val_accuracy: 0.6650
Epoch 4/50
40/40 [==============================] - 10s 249ms/step - loss: 0.5281 - accuracy: 0.7700 - val_loss: 5.2010 - val_accuracy: 0.6450
Epoch 5/50
40/40 [==============================] - 10s 258ms/step - loss: 0.4572 - accuracy: 0.8250 - val_loss: 7.6912 - val_accuracy: 0.6300
Epoch 6/50
40/40 [==============================] - 9s 222ms/step - loss: 0.5569 - accuracy: 0.8100 - val_loss: 7.3676 - val_accuracy: 0.6150
Epoch 7/50
40/40 [==============================] - 11s 266ms/step -

In [8]:
import os
print(os.listdir())
def img_to_array(path, category_no, X, y):
    for p in os.listdir(path):
        if p == ".DS_Store":
            continue
        #print(p)
        img = cv2.imread(os.path.join(path, p), cv2.IMREAD_GRAYSCALE)
        img_np = cv2.resize(img, dsize = (150, 150))
        X.append(img_np)
        y.append(category_no)

['.git', '.ipynb_checkpoints', 'model2.ipynb', 'Resources', 'saved_model.h5', 'static', 'templates', 'trainmodel.py', 'website.py']


In [26]:

import numpy as np
import cv2
path_test = 'Resources/chest_xray/test/'

category_list = ["NORMAL", "PNEUMONIA"]

X_test = []
y_test = []

for i in range(len(category_list)):
    img_to_array(path_test + category_list[i], i, X_test, y_test)
    
X_test = np.array(X_test).reshape(150,150)
y_test = np.array(y_test)

X_test = X_test / 255

ValueError: cannot reshape array of size 14040000 into shape (150,150)

In [ ]:
accuracy = model.evaluate(X_test, y_test)
print("Accuracy: %.2f" % (accuracy*100))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc = "upper left")
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc = "upper left")
plt.show()

In [ ]:
sns.heatmap(confusion_matrix(model.predict_classes(X_test), y_test), cmap = "viridis", annot = True)